<a href="https://colab.research.google.com/github/jimregan/effective-eureka/blob/mlm/MLM_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/awslabs/mlm-scoring

     |████████████████████████████████| 245kB 13.4MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 645kB 45.4MB/s 
     |████████████████████████████████| 3.8MB 54.1MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 1.1MB 47.0MB/s 


In [2]:
from transformers import *
model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
vocab = None

In [3]:
!pip install -q mxnet-cu101

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x3a88a000 @  0x7f5f35af8615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [4]:
from mlm.scorers import MLMScorer, MLMScorerPT
from mlm.models import get_pretrained
import mxnet
scorer = MLMScorerPT(model, vocab, tokenizer, [mxnet.gpu()])

/usr/local/lib/python3.6/dist-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [7]:
!wget http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz

--2020-09-02 20:15:10--  http://poleval.pl/task1/Poleval2020Task1Eval.tar.xz
Resolving poleval.pl (poleval.pl)... 213.135.36.94
Connecting to poleval.pl (poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12592244 (12M) [application/x-tar]
Saving to: ‘Poleval2020Task1Eval.tar.xz’

Poleval2020Task1Eva 100%[===================>]  12.01M  2.52MB/s    in 4.8s    

2020-09-02 20:15:16 (2.52 MB/s) - ‘Poleval2020Task1Eval.tar.xz’ saved [12592244/12592244]



In [8]:
!tar xf Poleval2020Task1Eval.tar.xz

In [ ]:
def one_best(id, sentences, scores, output):
  counter = 1
  lowest = 0.0
  lowest_number = 0
  for score in scores:
    if score < lowest:
      lowest = score
      lowest_number = counter
    counter = counter + 1
  output.write("%s-%d %s\n" % (id, lowest_number + 1, sentences[lowest_number]))

In [11]:
out = open(file="scored", mode="a", encoding="utf-8")
sentences = []
last_id = ''
counter = 1
with open(file="Poleval2020Task1Eval/nbest.txt", mode="r", encoding="utf-8") as nbest:
  for line in nbest:
    words = line.strip().split(' ')
    id = words[0]
    sentence = ' '.join(words[1:])
    score = scorer.score_sentences([sentence])
    out.write('%s\t%s\t%s\n' % (id, sentence, score))


RuntimeError: ignored

In [ ]:
!mv Poleval2020Task1Eval/nbest.txt nbest-last
!tail -n  $(($(wc -l nbest-last|awk '{print $1}') - $(wc -l scored|awk '{print $1}'))) nbest-last > Poleval2020Task1Eval/nbest.txt
